Load Video

1.   Load from Google Drive in Colab
2.   Make directory



In [0]:
# Google Colab import files in google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!cd drive; ls; 

In [1]:
!pip install tqdm
from tqdm import tqdm
import cv2 as cv
import numpy as np
import os
import datetime
from matplotlib import pyplot as plt

In [0]:
VIDEO_DIR = 'drive/FaceClustering/'
VIDEO_FILE_NAME = 'WINNER_EVERYDAY.mp4'
PROGRAM_START_TIME = datetime.datetime.now().strftime('%Y%m%d_%H-%M-%S')
SAVE_CROPPED_FACE_DIR = 'drive/Cropped_imgs/' + PROGRAM_START_TIME + '-' + VIDEO_FILE_NAME.split('.')[0]
CASCADE_DIR = 'drive/FaceClustering/lbpcascade_frontalface_improved.xml'

In [0]:

def make_directory(_directory_name):
    if not os.path.exists(_directory_name):
        os.makedirs(_directory_name)

def get_area_of_frame_face_recognition(img, face_cascade):
    grayed_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # def detectMultiScale(self, image, scaleFactor=None, minNeighbors=None, flags=None, minSize=None, maxSize=None)
    face_area = face_cascade.detectMultiScale(image=grayed_img,scaleFactor=1.3,minNeighbors=5)
    return face_area
  
def get_frame(video_capture, play_speed=1):
    curr_frame = video_capture.get(cv.CAP_PROP_POS_FRAMES)
    video_capture.set(cv.CAP_PROP_POS_FRAMES, curr_frame + play_speed)
    ret, img = video_capture.read()
    fps = video_capture.get(cv.CAP_PROP_FPS)
    return img, (curr_frame + play_speed) / fps, ret

def save_cropped_img(img, faces, sec):
    if len(faces) != 0:
        i = 0
        for (x, y, w, h) in faces:
            cv.imwrite(SAVE_CROPPED_FACE_DIR + "/" + str(sec).replace('.','_') + "_" + str(i) + ".jpg", img[y:y + h, x: x + w])
            i += 1
    if cv.waitKey(1) & 0xFF == ord('q'):
        return False
    else:
        return True

In [38]:
make_directory(SAVE_CROPPED_FACE_DIR)
m_video_capture = cv.VideoCapture(VIDEO_DIR + VIDEO_FILE_NAME)
m_face_cascade = cv.CascadeClassifier(CASCADE_DIR)
TOTAL_FRAME = m_video_capture.get(cv.CAP_PROP_FRAME_COUNT)
FRAME_STEP = 100
ALL = []
LOAD_FAIL = []
for frame in tqdm(range(0, int(TOTAL_FRAME), FRAME_STEP)):
    img, sec, ret = get_frame(video_capture=m_video_capture, play_speed=FRAME_STEP)
    ALL.append(sec)
    if not ret:
        LOAD_FAIL.append(sec)
        continue
    faces_area = get_area_of_frame_face_recognition(img=img, face_cascade=m_face_cascade)
    save_cropped_img(img, faces_area, sec)









  0%|          | 0/52 [00:00<?, ?it/s]







  2%|▏         | 1/52 [00:00<00:15,  3.22it/s]







  4%|▍         | 2/52 [00:00<00:14,  3.54it/s]







  6%|▌         | 3/52 [00:00<00:13,  3.55it/s]







  8%|▊         | 4/52 [00:01<00:12,  3.89it/s]







 10%|▉         | 5/52 [00:01<00:13,  3.53it/s]







 12%|█▏        | 6/52 [00:01<00:13,  3.31it/s]







 13%|█▎        | 7/52 [00:02<00:14,  3.16it/s]







 15%|█▌        | 8/52 [00:02<00:13,  3.14it/s]







 17%|█▋        | 9/52 [00:02<00:13,  3.25it/s]







 19%|█▉        | 10/52 [00:03<00:13,  3.00it/s]







 21%|██        | 11/52 [00:03<00:14,  2.89it/s]







 23%|██▎       | 12/52 [00:04<00:13,  2.86it/s]







 25%|██▌       | 13/52 [00:04<00:13,  2.90it/s]







 27%|██▋       | 14/52 [00:04<00:12,  2.99it/s]







 29%|██▉       | 15/52 [00:05<00:12,  2.91it/s]







 31%|███       | 16/52 [00:05<00:12,  2.88it/s]







 33%|███▎      | 17/52 [00:05<00:12,  2.88it/s]







 35%|███▍      | 18/

In [39]:
ALL, LOAD_FAIL

([4.170833333333333,
  8.425083333333333,
  12.679333333333332,
  16.93358333333333,
  21.18783333333333,
  25.442083333333333,
  29.69633333333333,
  33.950583333333334,
  38.20483333333333,
  42.45908333333333,
  46.71333333333333,
  50.96758333333333,
  55.22183333333333,
  59.47608333333333,
  63.73033333333333,
  67.98458333333333,
  72.23883333333333,
  76.49308333333333,
  80.74733333333333,
  85.00158333333333,
  89.25583333333333,
  93.51008333333333,
  97.76433333333333,
  102.01858333333332,
  106.27283333333332,
  110.52708333333332,
  114.78133333333332,
  119.03558333333332,
  123.28983333333332,
  127.54408333333332,
  131.79833333333332,
  136.05258333333333,
  140.30683333333332,
  144.56108333333333,
  148.8153333333333,
  153.06958333333333,
  157.3238333333333,
  161.57808333333332,
  165.8323333333333,
  170.08658333333332,
  174.3408333333333,
  178.59508333333332,
  182.84933333333333,
  187.10358333333332,
  191.35783333333333,
  195.61208333333332,
  199.866333

In [41]:
TOTAL_FRAME / m_video_capture.get(cv.CAP_PROP_FPS)

214.214